## 导入库

- **Pandas** => data manipulation
- **NumPy** => numerical computations
- **Matplotlib** and Seaborn => data visualization
- **scikit-learn** => machine learning tasks
- **os** => interacting with the operating system

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import os


## 数据交互

从指定的目录中读取 CSV 文件，将它们附加到 DataFrame(DataFrame) 列表中，连接到单个 DataFrame(df) ，并将其保存为 `datet.CSV`。

In [ ]:
dataframes = []
files = os.listdir('D:\\')
for file in files:
    if file.endswith('.csv') and file.startswith('DATA_'):
        filepath = os.path.join('D:\\', file)
        df = pd.read_csv(filepath)
        dataframes.append(df)

df = pd.concat(dataframes, ignore_index=True)
df.to_csv('dataset.csv', encoding='utf-8')


## 数据清理

Drop 掉时间戳的列：

In [ ]:
df.drop('timestamp', axis=1, inplace=True)

## 数据检查

提供有关 DataFrame 的形状、数据类型、汇总统计信息、缺失值(Null?)和目标变量的值计数的信息。

In [ ]:
print(df.shape)
print(df.info())
print(df.describe(include='all'))

if df.isnull().any() == True:
    print(df.isnull().sum())
    print('There are missing values in the data')

ValueCounts_target = df['target'].value_counts()
print(ValueCounts_target)

## 数据可视化

首先计算特征与目标变量之间的相关性，然后绘制相关性热力图来展示特征之间的相关性。Heatmap（热力图）使用颜色编码来表示不同特征之间的相关性程度。

In [ ]:
abs(df[df.columns[:]].corr()['target'][:].sort_values(ascending=False))      
corr = df.corr()

plt.figure(figsize=(17, 8))
sns.heatmap(corr, 
            xticklabels=corr.columns,
            linewidth=0.5,
            yticklabels=corr.columns,
            cmap="YlGnBu")
plt.show()

## 数据建模

将数据按照某种特定的规则进行分层处理，然后将每个分层（由特征向量组成）作为单独的数据集。最后，从原始的 DataFrame 中提取出特征矩阵 X 和目标向量 y，用于训练和测试模型。

In [ ]:
y = df['target']
X_np = df.values
layers = []
current_layer = []

for row in X_np:
    if row[-1] != 0x00:
        if current_layer:
            layers.append(np.array(current_layer))
        current_layer = [row]
    else:
        current_layer.append(row)

if current_layer:
    layers.append(np.array(current_layer))

X = np.array(layers)
X = X.drop('target', axis=1, inplace=True)

## 再次验证

验证数据处理的正确性，打印出第一个和第二个分层的特征矩阵的形状，以确保它们符合预期。

In [ ]:
print(X[0].shape, X[1].shape)

## 数据分割

将数据分割成训练集和测试集，并对特征数据进行标准化处理，以确保在模型训练过程中每个特征的重要性相同。

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)


## 训练模型

真正开始调用 PyTorch 的随机森林分类器模型，使用了 1000 棵决策树，并将训练好的模型保存在 `Rf_classifier` 中。

In [ ]:
Rf_classifier = RandomForestClassifier(n_estimators=1000)
Rf_classifier.fit(X_train_scaled, y_train)

## 评估模型

计算并输出随机森林分类器在测试集上的准确率和混淆矩阵。准确率用于评估模型的整体性能，而混淆矩阵则提供了模型在不同类别下的分类情况。

In [ ]:
y_pred_rf = Rf_classifier.predict(X_test_scaled)
accu_rf = accuracy_score(y_pred_rf, y_test)
print('Test Accuracy Random Forest Classifier:', accu_rf)
print('Confusion Matrix Random Forest Classifier:\n', confusion_matrix(y_pred_rf, y_test))